In [ ]:
import geopandas as gpd
import folium
import requests
from io import BytesIO

# 1. 병원 데이터 (WFS GeoJSON)
hospital_url = "https://geonode.themimu.info/geoserver/geonode/ows?service=WFS&version=1.0.0&request=GetFeature&outputFormat=application%2Fjson&typeName=geonode:health_facilities_myanmar2020_v20241016"
hospital_response = requests.get(hospital_url)
hospital_gdf = gpd.read_file(BytesIO(hospital_response.content))

# 2. 학교 데이터
school_gdf = gpd.read_file("./data/hotosm_mmr_education_facilities_points_geojson/hotosm_mmr_education_facilities_points_geojson.geojson")

# 3. 지도 초기화 (미얀마 중심)
m = folium.Map(location=[21.0, 96.0], zoom_start=6)

# 4. 병원 레이어
hospital_layer = folium.FeatureGroup(name="병원 (Hospitals)", show=True)

for _, row in hospital_gdf.iterrows():
    geom = row.geometry
    name = row.get("name", "Unknown")
    if geom and geom.geom_type == "Point":
        folium.CircleMarker(
            location=[geom.y, geom.x],
            radius=3,
            color='red',
            fill=True,
            fill_opacity=0.7,
            popup=name
        ).add_to(hospital_layer)

hospital_layer.add_to(m)

# 5. 학교 레이어
school_layer = folium.FeatureGroup(name="학교 (Schools)", show=True)

for _, row in school_gdf.iterrows():
    geom = row.geometry
    name = row.get("name", "Unknown")
    if geom and geom.geom_type == "Point":
        folium.CircleMarker(
            location=[geom.y, geom.x],
            radius=3,
            color='blue',
            fill=True,
            fill_opacity=0.7,
            popup=name
        ).add_to(school_layer)

school_layer.add_to(m)

# 6. Layer Control 추가 (토글 기능)
folium.LayerControl(collapsed=False).add_to(m)

# 7. 저장 또는 노트북에서 보기
m.save("myanmar_health_school_map.html")